# Instalando e testando o groq

In [ ]:
!pip install groq

Definição da key e teste de funcionalidade da API do Groq

In [ ]:
import os
os.environ['GROQ_API_KEY'] = "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"

In [ ]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models are critical in various applications, and their importance can be seen in the following aspects:

1. **Real-time Processing**: Many natural language processing (NLP) applications require fast processing to provide instant responses. For example, chatbots, virtual assistants, and language translation systems need to respond quickly to ensure a seamless user experience. Fast language models enable real-time processing, making them essential for such systems.

2. **Efficient Resource Utilization**: Fast language models can process large amounts of data quickly, which reduces the computational resources required. This leads to significant cost savings and more efficient resource utilization.

3. **Improved User Experience**: Fast language models can handle multiple tasks simultaneously, providing a better user experience. For instance, a fast language model can handle multiple conversations simultaneously, enabling a chatbot to handle a large volume of customer queries

# Definição da Classe Agente



* Contrutor (__init__): recebe o cliente da API e, opcionalmente, uma mensagem de sistema para definir contexto inicial.
* Chamada (call):  trata o agente como função — adiciona a mensagem do usuário ao histórico, invoca execute e retorna a resposta.
* Envio ao modelo (execute): envia todo o histórico de trocas para o modelo e devolve o completion gerado.



In [ ]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        chat_completion = self.client.chat.completions.create(
            messages=self.messages,
            model="llama3-70b-8192",
        )
        return chat_completion.choices[0].message.content

# Prompt



Responsavel por definir o comportamento que o Agente deve seguir. Nesse caso especifico um ciclo ReAct





* Thought: o agente explicita seu raciocínio interno
* Action: invoca uma tool (ferramenta)
* PAUSE/Observation: pausa enquanto executamos a ação e retorna o resultado  
* Answer: resposta final quando todas as ações forem concluídas



TESTE:


*   Foi adicionada a ferramenta surface_area_volume e todo o prompt (system_prompt) e o loop do agente foram atualizados para incluí-la nas ações disponíveis.



In [ ]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

Tools disponíveis:


*   calculate: operações matemáticas
*   get_planet_mass: recupera a massa de um planeta



In [ ]:
# tools
def calculate(operation: str) -> float:
    return eval(operation)

def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case"venus":
            return 4.867e24
        case _:
            return 0.0

# Execução do agente



*   O trecho inicializa um agente chamado neil_tyson com o cliente da API e um prompt de sistema
*   Uma pergunta é feita, e para responder ela são feitas observações externas, que são retornadas ao agente, até obter a resposta final
* Todo esse processo fica registrado (e é consultado em neil_tyson.messages)



In [ ]:
neil_tyson = Agent(client=client, system=system_prompt)

In [ ]:
result = neil_tyson("What is the mass of Earth times 5?")
print(result)

Thought: I need to find the mass of Earth.


In [ ]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [ ]:
result = neil_tyson()
print(result)

In [ ]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [ ]:
observation = get_planet_mass("Earth")
print(observation)

5.972e+24


In [ ]:
next_promt = f"Observation: {observation}"
result = neil_tyson(next_promt)
print(result)

Thought: I need to multiply this by 5


In [ ]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [ ]:
result = neil_tyson()
print(result)

In [ ]:
observation = calculate("3.285e23 * 5")
print(observation)

1.6425e+24


In [ ]:
next_promt = f"Observation: {observation}"
result = neil_tyson(next_promt)
print(result)

Answer: The mass of Earth times 5 is 1.98625e+25.


In [ ]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

# Execução do agente (em loop)



É definido um agent_loop, que é resposável por:


* Fazer chamadas ao agente
* Executar ferramentas externas quando detecta comandos de ação (injetando os resultados como Observation)
* Esse loop para ao encontrar answer ou atingir o número máximo de iterações





In [ ]:
import re

def agent_loop(max_iterations, system, query):
    agent = Agent(client, system_prompt)
    tools = ['calculate', 'get_planet_mass']
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"
            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break

In [ ]:
agent_loop(max_iterations=10, system=system_prompt, query="What is the mass of the Earth plus the mass of Mercury, and all of it times 5")

Thought: I need to find the mass of Earth and Mercury, then add them together and finally multiply the result by 5.
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now I have the mass of Earth, but I still need to get its mass.
Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: Now I have the masses of both Earth and Mercury, I need to add them together.
Action: calculate: 5.972e24 + 3.285e+23
PAUSE
Observation: 6.300500000000001e+24
Thought: Now I have the total mass, but I still need to multiply it by 5.
Action: calculate: 6.300500000000001e+24 * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have the final result.

Answer: The mass of the Earth plus the mass of Mercury, and all of it times 5 is 3.1502500000000004e+25.
